LASSO

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero
from data_generator import gendata_Linear_ed
import torch
from log_hazard_binary import g_dnn_bin
from log_linear_binary import E_ols_bin
from log_linear_binary import E_lasso_bin
from optITR import optITR_lin_bin
from optITR import optITR_lin_bined
import time
#As an example for linear case
# patiences = 10
# n_node = 50
# n_layer = 1
# t_nodes = 100
# t_fig = 50
# n_lr = 1e-3
# n_epoch = 1000
# s_k = 0.1*np.arange(100)  #only focus on interval [0,10]
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop = 200

missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Linear_ed(500, 0.5,beta,gamma)

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')    
    val_data = gendata_Linear_ed(200, 0.5,beta,gamma)
    train_data = gendata_Linear_ed(800, 0.5,beta,gamma)
    St = E_lasso_bin(train_data, val_data, test_data,lam_min=-2,lam_max=0,K=5)
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_lin_bined(test_data,Cmean_R_X_treated, Cmean_R_X_untreat, beta,gamma)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.054200000000000005
0.01827347804880067
0.0029788346613734383
0.002230220796645766


In [3]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero
from data_generator import gendata_Deep_ed
import torch
from log_hazard_binary import g_dnn_bin
from log_linear_binary import E_ols_bin
from log_linear_binary import E_lasso_bin
from optITR import optITR_lin_bin
from optITR import optITR_deep_bin
import time
loop = 200

missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep_ed(500, 0.5)

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r') 
    val_data = gendata_Deep_ed(200, 0.5)
    train_data = gendata_Deep_ed(800, 0.5)
    St = E_lasso_bin(train_data, val_data, test_data,lam_min=-2,lam_max=0,K=5)
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_deep_bin(test_data,Cmean_R_X_treated, Cmean_R_X_untreat)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.31523
0.01806397243133415
0.10108280049805451
0.012689699914405148


RDL

In [4]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression,LassoCV
from sklearn.preprocessing import StandardScaler
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero 
from data_generator import gendata_Linear_ed
from log_dlearn_binary import RDL_bin
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop=200
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Linear_ed(500, 0.5,beta,gamma)
mu_test = test_data['mu0']
epsilon_test = test_data['epsilon']

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')    
    val_data = gendata_Linear_ed(200, 0.5,beta,gamma)
    train_data = gendata_Linear_ed(800, 0.5,beta,gamma)
    res_rdl = RDL_bin(train_data,val_data,test_data,alphas=np.logspace(-3,0,10),precompute=False)
    d_est = res_rdl['itr_test']
    X_test = test_data['X']
    d_opt = X_test[:,0:5]@(gamma-beta)>0 
    d_opt = d_opt.astype(int)
    missrate[i] = np.mean(np.abs(d_opt-d_est))
    #quality value 
    theta = np.vstack((beta,gamma))
    quality = X_test[:,0:5]@theta.T
    d_est1 = np.vstack((1-d_est,d_est)) #untreat-treated
    d_opt1 = np.vstack((1-d_opt,d_opt))

    V_est = np.mean(np.exp(np.sum(d_est1.T*quality,axis=1)+mu_test+epsilon_test))

    V_opt = np.mean(np.exp(np.sum(d_opt1.T*quality,axis=1)+mu_test+epsilon_test))
    
    Regret = V_opt - V_est
    regret[i] = Regret
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.08221
0.021990359251271907
0.008217886909263778
0.005370103879783088


In [6]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression,LassoCV
from sklearn.preprocessing import StandardScaler
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero 
from data_generator import gendata_Deep_ed
from log_dlearn_binary import RDL_bin
# beta = np.array([0,-0.5,0.5,-0.25,0.5])
# gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop=200
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep_ed(500, 0.5)
mu_test = test_data['mu0']
epsilon_test = test_data['epsilon']
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Deep_ed(200, 0.5)
    train_data = gendata_Deep_ed(800, 0.5)
    res_rdl = RDL_bin(train_data,val_data,test_data,alphas=np.logspace(-3,0,10),precompute=False)
    d_est = res_rdl['itr_test']
    X = test_data['X']
    g_X = test_data['g_X']
    h_X = test_data['h_X']
    # g_X =  np.cos(X[:,0]**2+2*X[:,1]**2+X[:,2]**3+np.sqrt(X[:,3]+1)*np.log(X[:,4]+2)/20)
        #A=1, influence h_X
    # h_X = X[:,0]/3 + np.exp(X[:,1])/4 + np.cos(X[:,2]* X[:,3])\
                # - (np.log(X[:,4]+2))/3 -1
    d_opt = (h_X-g_X)>0
    d_opt = d_opt.astype(int)
    missrate[i] = np.mean(np.abs(d_opt-d_est))

    
    quality = (np.vstack((g_X , h_X))).T
    d_est1 = np.vstack((1-d_est,d_est)) #untreat-treated
    d_opt1 = np.vstack((1-d_opt,d_opt))

    V_est = np.mean(np.exp(np.sum(d_est1.T*quality,axis=1)+mu_test+epsilon_test))

    V_opt = np.mean(np.exp(np.sum(d_opt1.T*quality,axis=1)+mu_test+epsilon_test))
    Regret = V_opt - V_est
    regret[i] = Regret
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))


Results:e: 200
0.32199
0.017491137756018047
0.10742164244422607
0.01238428332112072


SDL

In [15]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression,LassoCV
from sklearn.preprocessing import StandardScaler
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero
from data_generator import gendata_Linear_ed
from log_dlearn_binary import SDL_bin
import warnings
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop=200
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Linear_ed(500, 0.5,beta,gamma)
mu_test = test_data['mu0']
epsilon_test = test_data['epsilon']
warnings.filterwarnings('ignore')
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Linear_ed(200, 0.5,beta,gamma)
    train_data = gendata_Linear_ed(800, 0.5,beta,gamma)
    res_sdl = SDL_bin(train_data,val_data,test_data,alphas=np.logspace(-3,0,10),precompute=False)
    d_est = res_sdl['itr_test']
    X_test = test_data['X']
    d_opt = X_test[:,0:5]@(gamma-beta)>0 
    d_opt = d_opt.astype(int)
    missrate[i] = np.mean(np.abs(d_opt-d_est))
    #quality value 
    theta = np.vstack((beta,gamma))
    quality = X_test[:,0:5]@theta.T
    d_est1 = np.vstack((1-d_est,d_est)) #untreat-treated
    d_opt1 = np.vstack((1-d_opt,d_opt))

    V_est = np.mean(np.exp(np.sum(d_est1.T*quality,axis=1)+mu_test+epsilon_test))

    V_opt = np.mean(np.exp(np.sum(d_opt1.T*quality,axis=1)+mu_test+epsilon_test))
    
    Regret = V_opt - V_est
    regret[i] = Regret
    # time.sleep(1)
print('Results:                      ')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))


Results:                      
0.11456000000000001
0.029496548950682346
0.017988813885627446
0.009694439849195066


In [16]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression,LassoCV
from sklearn.preprocessing import StandardScaler
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero 
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_ed 
from log_dlearn_binary import SDL_bin
import warnings
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop=200
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep_ed(500, 0.5)
mu_test = test_data['mu0']
epsilon_test = test_data['epsilon']
warnings.filterwarnings('ignore')
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')  
    val_data = gendata_Deep_ed(200, 0.5)
    train_data = gendata_Deep_ed(800, 0.5)
    res_sdl = SDL_bin(train_data,val_data,test_data,alphas=np.logspace(-3,0,10),precompute=False)
    d_est = res_sdl['itr_test']
    X = test_data['X']
    g_X = test_data['g_X']
    h_X = test_data['h_X']
    # g_X =  np.cos(X[:,0]**2+2*X[:,1]**2+X[:,2]**3+np.sqrt(X[:,3]+1)*np.log(X[:,4]+2)/20)
        #A=1, influence h_X
    # h_X = X[:,0]/3 + np.exp(X[:,1])/4 + np.cos(X[:,2]* X[:,3])\
                # - (np.log(X[:,4]+2))/3 -1
    d_opt = (h_X-g_X)>0
    d_opt = d_opt.astype(int)
    missrate[i] = np.mean(np.abs(d_opt-d_est))
    
    quality = (np.vstack((g_X , h_X))).T
    d_est1 = np.vstack((1-d_est,d_est)) #untreat-treated
    d_opt1 = np.vstack((1-d_opt,d_opt))

    V_est = np.mean(np.exp(np.sum(d_est1.T*quality,axis=1)+mu_test+epsilon_test))

    V_opt = np.mean(np.exp(np.sum(d_opt1.T*quality,axis=1)+mu_test+epsilon_test))
    Regret = V_opt - V_est
    regret[i] = Regret
    # regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:                   ')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))


Results:                   
0.3148699999999999
0.01743166945533331
0.10343056892946911
0.015255797231830345


DL

In [7]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression,LassoCV
from sklearn.preprocessing import StandardScaler
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero 
from data_generator import gendata_Linear_ed
from log_dlearn_binary import DL_bin
import warnings
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop=200
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Linear_ed(500, 0.5,beta,gamma)
mu_test = test_data['mu0']
epsilon_test = test_data['epsilon']
warnings.filterwarnings('ignore')
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Linear_ed(200, 0.5,beta,gamma)
    train_data = gendata_Linear_ed(800, 0.5,beta,gamma)
    res_sdl = DL_bin(train_data,val_data,test_data,alphas=np.logspace(-3,0,10),precompute=False)
    d_est = res_sdl['itr_test']
    X_test = test_data['X']
    d_opt = X_test[:,0:5]@(gamma-beta)>0 
    d_opt = d_opt.astype(int)
    missrate[i] = np.mean(np.abs(d_opt-d_est))
    #quality value 
    theta = np.vstack((beta,gamma))
    quality = X_test[:,0:5]@theta.T
    d_est1 = np.vstack((1-d_est,d_est)) #untreat-treated
    d_opt1 = np.vstack((1-d_opt,d_opt))

    V_est = np.mean(np.exp(np.sum(d_est1.T*quality,axis=1)+mu_test+epsilon_test))

    V_opt = np.mean(np.exp(np.sum(d_opt1.T*quality,axis=1)+mu_test+epsilon_test))
    
    Regret = V_opt - V_est
    regret[i] = Regret
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.10065000000000002
0.02804277981941163
0.014451750932255899
0.008753954303702212


In [8]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, LogisticRegression,LassoCV
from sklearn.preprocessing import StandardScaler
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero 
from data_generator import gendata_Deep_ed
from log_dlearn_binary import DL_bin
import warnings
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop=200
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep_ed(500, 0.5)
mu_test = test_data['mu0']
epsilon_test = test_data['epsilon']
warnings.filterwarnings('ignore')
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')  
    val_data = gendata_Deep_ed(200, 0.5)
    train_data = gendata_Deep_ed(800, 0.5)
    res_sdl = DL_bin(train_data,val_data,test_data,alphas=np.logspace(-3,0,10),precompute=False)
    d_est = res_sdl['itr_test']
    X = test_data['X']
    g_X = test_data['g_X']
    h_X = test_data['h_X']
    # g_X =  np.cos(X[:,0]**2+2*X[:,1]**2+X[:,2]**3+np.sqrt(X[:,3]+1)*np.log(X[:,4]+2)/20)
        #A=1, influence h_X
    # h_X = X[:,0]/3 + np.exp(X[:,1])/4 + np.cos(X[:,2]* X[:,3])\
                # - (np.log(X[:,4]+2))/3 -1
    d_opt = (h_X-g_X)>0
    d_opt = d_opt.astype(int)
    missrate[i] = np.mean(np.abs(d_opt-d_est))
    
    quality = (np.vstack((g_X , h_X))).T
    d_est1 = np.vstack((1-d_est,d_est)) #untreat-treated
    d_opt1 = np.vstack((1-d_opt,d_opt))

    V_est = np.mean(np.exp(np.sum(d_est1.T*quality,axis=1)+mu_test+epsilon_test))

    V_opt = np.mean(np.exp(np.sum(d_opt1.T*quality,axis=1)+mu_test+epsilon_test))
    Regret = V_opt - V_est
    regret[i] = Regret
    # regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))


Results:e: 200
0.31749999999999995
0.027620101375628592
0.10735686047711422
0.021118849213615804


OLS

In [9]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero
from data_generator import gendata_Linear_ed
import torch
from log_hazard_binary import g_dnn_bin
from log_linear_binary import E_ols_bin
from optITR import optITR_lin_bin
from optITR import optITR_lin_bined
import time
#As an example for linear case
# patiences = 10
# n_node = 50
# n_layer = 1
# t_nodes = 100
# t_fig = 50
# n_lr = 1e-3
# n_epoch = 1000
# s_k = 0.1*np.arange(100)  #only focus on interval [0,10]
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop = 200

missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Linear_ed(500, 0.5,beta,gamma)

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Linear_ed(200, 0.5,beta,gamma)
    train_data = gendata_Linear_ed(800, 0.5,beta,gamma)
    St = E_ols_bin(train_data, val_data, test_data)
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_lin_bined(test_data,Cmean_R_X_treated, Cmean_R_X_untreat, beta,gamma)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.08277
0.019654696639734737
0.009088802517304513
0.005584576638827093


In [10]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero
from data_generator import gendata_Deep_ed
import torch
from log_hazard_binary import g_dnn_bin
from log_linear_binary import E_ols_bin
from optITR import optITR_lin_bin
from optITR import optITR_deep_bin
import time
loop = 200

missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep_ed(500, 0.5)

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')
    val_data = gendata_Deep_ed(200, 0.5)
    train_data = gendata_Deep_ed(800, 0.5)
    St = E_ols_bin(train_data, val_data, test_data)
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_deep_bin(test_data,Cmean_R_X_treated, Cmean_R_X_untreat)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.32343999999999995
0.015231756300571516
0.1057630358331249
0.010501255897592768


DLS

In [11]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero
from data_generator import gendata_Linear_ed
import torch
from log_dls_binary import E_dls_bin
from optITR import optITR_lin_bin
from optITR import optITR_deep_bin
from optITR import optITR_lin_bined
import time
loop = 200
patiences = 20
n_node = 64
n_layer = 1
n_lr = 1e-3
n_epoch = 1000
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
test_data = gendata_Linear_ed(500, 0.5,beta,gamma)
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r') 
    val_data = gendata_Linear_ed(200, 0.5,beta,gamma)
    train_data = gendata_Linear_ed(800, 0.5,beta,gamma)
    St = E_dls_bin(train_data, val_data, test_data, n_layer, n_node, n_lr, n_epoch, patiences,show_val=False)
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_lin_bined(test_data,Cmean_R_X_treated, Cmean_R_X_untreat,beta,gamma)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.10710000000000001
0.021678330193997876
0.014996658673501265
0.006784294294629767


In [12]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Deep_hetero
from data_generator import gendata_Deep_ed
import torch
from log_dls_binary import E_dls_bin
from optITR import optITR_lin_bin
from optITR import optITR_deep_bin
import time
loop = 200
patiences = 20
n_node = 64
n_layer = 2
n_lr = 1e-3
n_epoch = 1000
missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep_ed(500, 0.5)
for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r') 
    val_data = gendata_Deep_ed(200, 0.5)
    train_data = gendata_Deep_ed(800, 0.5)
    St = E_dls_bin(train_data, val_data, test_data, n_layer, n_node, n_lr, n_epoch, patiences,show_val=False)
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_deep_bin(test_data,Cmean_R_X_treated, Cmean_R_X_untreat)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:e: 200
0.19927999999999998
0.06332615257537758
0.05365755330397704
0.03207119341200661


Proposed

In [20]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Linear_ed
from data_generator import gendata_Deep_hetero
import torch
from log_hazard_binary import g_dnn_bin
from optITR import optITR_lin_bin
from optITR import optITR_lin_bined
import time
#As an example for linear case
set_seed(114)
patiences = 30
n_node = 64
n_layer = 2
t_nodes = 100
t_fig = 50
n_lr = 1e-4
n_epoch = 1000
s_k = 0.1*np.arange(100)  #only focus on interval [0,10]
beta = np.array([0,-0.5,0.5,-0.25,0.5])
gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop = 200

missrate = np.zeros(loop)
regret = np.zeros(loop)

test_data = gendata_Linear_ed(500, 0.5,beta,gamma)

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r')   
    val_data = gendata_Linear_ed(200, 0.5,beta,gamma)
    train_data = gendata_Linear_ed(800, 0.5,beta,gamma)
    St = g_dnn_bin(train_data, val_data, test_data, t_nodes, t_fig, s_k, n_layer, n_node, n_lr, n_epoch, patiences,show_val=False)
    S_R_X_ibs = St['S_R_X_untreat']
    S_R_X_ibs1 = St['S_R_X_treated']
    Lambda_R_X_untreat = St['Lambda_R_X_untreat']
    Lambda_R_X_treated = St['Lambda_R_X_treated']
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_lin_bined(test_data,Cmean_R_X_treated, Cmean_R_X_untreat, beta,gamma)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:                           ')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:                           
0.10144000000000002
0.03720277409011323
0.013618087429908972
0.01612126670414663


In [17]:
import numpy as np
import numpy.random as ndm
import matplotlib.pyplot as plt
from seed import set_seed
from data_generator import gendata_Linear_hetero
from data_generator import gendata_Deep_hetero
from data_generator import gendata_Deep_ed
import torch
from log_hazard_binary import g_dnn_bin
from optITR import optITR_deep_bin
import time
#As an example for linear case
patiences = 30
n_node = 64
n_layer = 2
t_nodes = 100
t_fig = 50
n_lr = 2.5e-3
n_epoch = 1000
s_k = 0.1*np.arange(100)  #only focus on interval [0,10]
# beta = np.array([0,-0.5,0.5,-0.25,0.5])
# gamma = np.array([0.5,-0.25,0,0.5,-0.25])
loop = 200

missrate = np.zeros(loop)
regret = np.zeros(loop)
set_seed(114)
test_data = gendata_Deep_ed(500, 0.5)

for i in range(loop):
    set_seed(1145+i)
    print(f"Loop time: {i+1}", end='\r') 
    val_data = gendata_Deep_ed(200, 0.5)
    train_data = gendata_Deep_ed(800, 0.5)
    St = g_dnn_bin(train_data, val_data, test_data, t_nodes, t_fig, s_k, n_layer, n_node, n_lr, n_epoch, patiences,show_val=False)
    S_R_X_ibs = St['S_R_X_untreat']
    S_R_X_ibs1 = St['S_R_X_treated']
    Lambda_R_X_untreat = St['Lambda_R_X_untreat']
    Lambda_R_X_treated = St['Lambda_R_X_treated']
    Cmean_R_X_untreat = St['Cmean_R_X_untreat']
    Cmean_R_X_treated = St['Cmean_R_X_treated']
    ditr = optITR_deep_bin(test_data,Cmean_R_X_treated, Cmean_R_X_untreat)
    missrate[i] = ditr['mis_rate']
    regret[i] = ditr['Regret']
    # time.sleep(1)
print('Results:                                  ')
print(np.mean(missrate))
print(np.std(missrate))
print(np.mean(regret))
print(np.std(regret))

Results:                                  
0.18664000000000003
0.02608122696500301
0.04705707692510426
0.011148816246179966
